![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#nltk.download('all')

In [3]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import pickle
import joblib

from keras.preprocessing import sequence
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import nltk

#nltk.download('omw')


from xgboost import XGBClassifier

In [4]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('dataTraining.csv', index_col=0)
dataTesting = pd.read_csv('dataTesting.csv', index_col=0)

In [5]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [6]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [7]:
#Pasar palabras a minusculas
def minusculizar(df):
    df2 = df.copy()
    df2['title'] = df2['title'].apply(lambda x : x.lower())
    df2['plot'] = df2['plot'].apply(lambda x : x.lower())
    return df2

dataTraining_min  = minusculizar(dataTraining)
dataTraining_min

,year,title,plot,genres,rating
3107,2003,most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,how to be a serial killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,a woman's face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,executive suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,narrow margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6
...,...,...,...,...,...
8417,2010,our family wedding,""" our marriage , their wedding . "" it ' s l...","['Comedy', 'Romance']",4.9
1592,1984,conan the destroyer,"the wandering barbarian , conan , alongside ...","['Action', 'Adventure', 'Fantasy']",5.8
1723,1955,kismet,"like a tale spun by scheherazade , kismet fol...","['Adventure', 'Musical', 'Fantasy', 'Comedy', ...",6.4
7605,1982,the secret of nimh,"mrs . brisby , a widowed mouse , lives in a...","['Animation', 'Adventure', 'Drama', 'Family', ...",7.6


Plots

In [8]:
Listadeplots = dataTraining_min['plot'].tolist()
lista_palabras=[]
for i in Listadeplots:
    lista_palabras.extend(i.split())
palabras_unicas_plot = list(set(lista_palabras))

stopwords_english = nltk.corpus.stopwords.words('english')

lista_palabras_completa_filtrada = [palabra for palabra in lista_palabras if palabra.lower() not in stopwords_english]
lista_filtrada_plot = set(lista_palabras_completa_filtrada)

len(palabras_unicas_plot),len(lista_filtrada_plot)

(38734, 38584)

Creacion del diccionario de lematización

In [9]:
# Inicializar el lematizador
lemmatizer = WordNetLemmatizer()

# Mapear las etiquetas POS de NLTK a las etiquetas POS de WordNet
tag_map = {
    'N': nltk.corpus.wordnet.NOUN,
    'V': nltk.corpus.wordnet.VERB,
    'R': nltk.corpus.wordnet.ADV,
    'J': nltk.corpus.wordnet.ADJ
}

# Lematizar la lista de palabras
lista_lematizada = []
for palabra in lista_filtrada_plot:
    # Obtener la etiqueta POS de cada palabra
    pos_tag = nltk.pos_tag([palabra])[0][1][0].upper()
    # Mapear la etiqueta POS a las etiquetas POS de WordNet
    pos_tag = tag_map.get(pos_tag, nltk.corpus.wordnet.NOUN)
    
    # Lematizar la palabra
    lema = lemmatizer.lemmatize(palabra, pos=pos_tag)
    lista_lematizada.append(lema)

# Imprimir la lista de palabras lematizada
print(lista_lematizada)

diccionario_original_a_lemas = {clave: valor for clave, valor in zip(lista_filtrada_plot, lista_lematizada)}


lista_unicos_lematizada = list(set(lista_lematizada))
len(lista_unicos_lematizada)

['fangoria', 'haired', 'confess', 'sternwood', 'bag', 'leprosy', 'likelihood', 'eddie', 'farthest', 'distillery', 'primetime', 'jamesey', 'broil', 'mishima', 'hoppus', 'tell', 'beering', 'winner', 'freeway', 'aide', 'waste', 'wallet', 'dim', 'jagjit', 'trinke', 'corporation', 'sumptuous', 'illusionist', 'surreptitiously', 'macedonian', 'knock', 'scrawl', 'disapprove', 'tormentor', 'milligan', 'hitch', 'tyne', 'fragile', 'exterminate', 'cogan', 'player', 'stealthily', 'cartwright', 'juvenile', 'deprima', 'comprehension', 'deon', 'oceanfront', 'vero', 'cathars', 'talisman', 'judi', 'argentinian', 'air', 'throughs', 'riche', 'chastise', 'babylon', 'bannon', 'longreach', 'unwisely', 'dam', 'pallbearer', 'unsophisticated', 'mackintosh', 'snn', 'paranoiac', 'irregularity', 'butter', 'auditor', 'reinvention', 'sedgwick', 'wedge', 'christopher', 'holmes', 'luchador', 'whore', 'radically', 'safeguard', 'smaug', 'animator', 'mcginnis', 'kenji', 'inexcusable', 'trogs', 'shue', 'reconnection', 'in

31371

In [10]:
diccionario_original_a_lemas

{'fangoria': 'fangoria',
 'haired': 'haired',
 'confess': 'confess',
 'sternwood': 'sternwood',
 'bagged': 'bag',
 'leprosy': 'leprosy',
 'likelihood': 'likelihood',
 'eddie': 'eddie',
 'farthest': 'farthest',
 'distillery': 'distillery',
 'primetime': 'primetime',
 'jamesey': 'jamesey',
 'broiled': 'broil',
 'mishima': 'mishima',
 'hoppus': 'hoppus',
 'telling': 'tell',
 'beering': 'beering',
 'winner': 'winner',
 'freeway': 'freeway',
 'aides': 'aide',
 'waste': 'waste',
 'wallet': 'wallet',
 'dim': 'dim',
 'jagjit': 'jagjit',
 'trinke': 'trinke',
 'corporations': 'corporation',
 'sumptuous': 'sumptuous',
 'illusionist': 'illusionist',
 'surreptitiously': 'surreptitiously',
 'macedonian': 'macedonian',
 'knocked': 'knock',
 'scrawl': 'scrawl',
 'disapproving': 'disapprove',
 'tormentors': 'tormentor',
 'milligan': 'milligan',
 'hitches': 'hitch',
 'tyne': 'tyne',
 'fragile': 'fragile',
 'exterminating': 'exterminate',
 'cogan': 'cogan',
 'player': 'player',
 'stealthily': 'stealthily

Palabras lematizadas más comunes

In [11]:
df_Listado_completo_filtrado_no_stopwords = pd.DataFrame(lista_palabras_completa_filtrada,columns=['words'])
df_Listado_completo_filtrado_no_stopwords['Lematized'] = df_Listado_completo_filtrado_no_stopwords['words'].apply(lambda x : diccionario_original_a_lemas[x])
df_Listado_completo_filtrado_no_stopwords['Count'] = 1
Repeticiones_por_lematized = df_Listado_completo_filtrado_no_stopwords[['Lematized','Count']].groupby('Lematized').sum().reset_index().sort_values('Count',ascending=False)
filtro = Repeticiones_por_lematized['Lematized'].apply(lambda x: len(x) > 1)
resultados_filtrados = Repeticiones_por_lematized[filtro].reset_index(drop=True)
Total_palabras = resultados_filtrados['Count'].sum()
resultados_filtrados['Part'] = resultados_filtrados['Count'].apply(lambda x: x/Total_palabras)
resultados_filtrados['Acumulado'] = resultados_filtrados['Part'].cumsum()
Porcentaje = 0.8
pareto = resultados_filtrados[resultados_filtrados['Acumulado']<=Porcentaje].shape
print('El % de las palabras se encuentra en las primeras ' + str(pareto[0]))

Palabras_Para_Clasificar_Lematizadas = resultados_filtrados[resultados_filtrados['Acumulado']<=Porcentaje]['Lematized'].tolist()

vocabulario_lematizado = {palabra: indice for indice, palabra in enumerate(Palabras_Para_Clasificar_Lematizadas)}

resultados_filtrados.head(10)

El 95% de las palabras se encuentra en las primeras 3535


,Lematized,Count,Part,Acumulado
0,life,3640,0.006784,0.006784
1,one,3068,0.005718,0.012502
2,get,3068,0.005718,0.018219
3,find,2875,0.005358,0.023577
4,go,2405,0.004482,0.028059
5,new,2271,0.004232,0.032292
6,take,2240,0.004175,0.036466
7,friend,2190,0.004081,0.040548
8,year,2085,0.003886,0.044434
9,make,2057,0.003834,0.048267


Lematizar texto original y eliminar stopwords

In [12]:
def lematizar_texto(texto, diccionario):
    palabras = texto.split()  # Dividir el texto en palabras
    palabras_lematizadas = [diccionario.get(palabra, palabra) for palabra in palabras]  # Obtener las palabras lematizadas del diccionario
    palabras_filtradas = [palabra for palabra in palabras_lematizadas if palabra.lower() not in stopwords_english]  # Filtrar las stopwords
    texto_lematizado = ' '.join(palabras_filtradas)  # Unir las palabras lematizadas en un nuevo texto
    return texto_lematizado

dataTraining_min['plot_lematized'] = dataTraining_min['plot'].apply(lambda x: lematizar_texto(x,diccionario_original_a_lemas))
dataTraining_min.head()


,year,title,plot,genres,rating,plot_lematized
3107,2003,most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0,story single father take eight year - old son ...
900,2008,how to be a serial killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6,serial killer decides teach secret satisfy car...
6724,1941,a woman's face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2,"sweden , female blackmailer disfigure facial s..."
4704,1954,executive suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4,"friday afternoon new york , president tredway ..."
2582,1990,narrow margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6,"los angeles , editor publishing house carol hu..."


In [13]:
vectorizer = CountVectorizer(vocabulary=vocabulario_lematizado)
xplot_vectorizer = vectorizer.transform(dataTraining_min['plot_lematized'])#.todense()
features = vectorizer.get_feature_names()
prueba = dataTraining_min['plot_lematized'].tolist()[2].split()
print(xplot_vectorizer.shape)
xplot_vectorizer

(7895, 3535)


<7895x3535 sparse matrix of type '<class 'numpy.int64'>'
	with 351484 stored elements in Compressed Sparse Row format>

Plot con contador de letras

In [14]:
from keras.preprocessing import sequence

In [15]:
X = dataTraining_min['plot'].tolist()
# Definición de vocabulario
voc = set(''.join(X))
vocabulary = {x: idx + 1 for idx, x in enumerate(set(voc))}
vocabulary

# Definición máximo largo de embedding
max_url_len = 500

# Codificación (embedding) de la URL con el vocabulario definido anteriormente
X = [x[:max_url_len] for x in X]
X = [[vocabulary[x1] for x1 in x if x1 in vocabulary.keys()] for x in X]


In [16]:
X_pad = sequence.pad_sequences(X, maxlen=max_url_len)
X_pad.shape

(7895, 500)

Variable de interes

In [17]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])
y_genres

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0]])

In [18]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(xplot_vectorizer, y_genres, test_size=0.33, random_state=42)

Red neuronal recurrente

In [19]:
y_test_genres.shape,X_train.shape

((2606, 24), (5289, 3535))

In [20]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from livelossplot import PlotLossesKeras

In [21]:
from keras import backend as K

# Limpia la sesión actual de Keras
K.clear_session()

In [30]:
# Definición red neuronal con la función Sequential()
model = Sequential()

# Definición de la capa embedding
model.add(Embedding(len(vocabulario_lematizado) + 1, 1200, input_length=xplot_vectorizer.shape[1]))
# Definición de la capa recurrente LSTM
model.add(LSTM(600))
# Definición de dropout para evitar overfitting
model.add(Dropout(0.5))
# Definición capa densa con función sigmoide para predicción binaria final
model.add(Dense(y_test_genres.shape[1], activation='sigmoid'))

# Definición de función de perdida.
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Impresión de la arquitectura de la red neuronal
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3535, 1200)        4243200   
                                                                 
 lstm_1 (LSTM)               (None, 600)               4322400   
                                                                 
 dropout_1 (Dropout)         (None, 600)               0         
                                                                 
 dense_1 (Dense)             (None, 24)                14424     
                                                                 
Total params: 8,580,024
Trainable params: 8,580,024
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_train_dense= X_train.toarray()
X_test_dense= X_test.toarray()

In [ ]:
model.fit(X_train_dense, y_train_genres, validation_data=(X_test_dense, y_test_genres), 
          batch_size=128, epochs=10, verbose=1,
          callbacks=[PlotLossesKeras()])

Epoch 1/10


transformación de conjunto de test

In [ ]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

In [ ]:
clf = XGBClassifier(n_jobs=-1)
clf.fit(X_train,y_train_genres)

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

clfxbgoost = XGBClassifier()

grid_search = GridSearchCV(estimator=clfxbgoost, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train, y_train_genres)

best_params = grid_search.best_params_
best_score = grid_search.best_score_